In [27]:
:dep nsi = { package = "nsi", path = "/home/moritz/code/crates/nsi-old", features = ["jupyter"] }
:dep p_ops = { package = "polyhedron-ops", path = "/home/moritz/code/crates/polyhedron-ops", features = [ "nsi" ] }

Remove `nsi` namespace prefix.

In [28]:
use nsi::*;

Create a rendering [context](https://docs.rs/nsi/latest/nsi/struct.Context.html).

In [29]:
let ctx = nsi::Context::new(None).unwrap();

Create a dodecahedron.

In [30]:
let polyhedron = p_ops::Polyhedron::dodecahedron();

We convert the dodecahedron into an NSI node and connect it to the scene’s root.

In [31]:
let handle = polyhedron.to_nsi(&ctx, None, None, None, None);
ctx.connect(&handle, None, ROOT, "objects", None);

Error: mismatched types

Set up an [attribute](https://nsi.readthedocs.io/en/latest/nodes.html#node-attributes) to hook up a shader to our geometry node.

We then create & connect a [shader](https://nsi.readthedocs.io/en/latest/nodes.html#the-shader-node) and instance a *principled* [OSL](https://opensource.imageworks.com/osl.html) shader.

In [26]:
ctx.create("dodeca_attrib", ATTRIBUTES, None);
ctx.connect("dodeca_attrib", None, handle.as_str(), "geometryattributes", None);

ctx.create("dodeca_shader", SHADER, None);
ctx.connect("dodeca_shader", None, "dodeca_attrib", "surfaceshader", None);

ctx.set_attribute(
    "dodeca_shader",
    &[
        string!("shaderfilename", "${DELIGHT}/osl/dlPrincipled"),
        color!("i_color", &[0.8, 0.5, 0.3]),
        float!("roughness", 0.1),
        float!("specular_level", 0.5),
        float!("metallic", 1.0),
    ],
);

Error: cannot find value `handle` in this scope

Next we set up an [environment](https://nsi.readthedocs.io/en/latest/nodes.html#the-environment-node) node and instance an emissive *environment* [OSL](https://opensource.imageworks.com/osl.html) shader.

In [14]:
ctx.create("environment", ENVIRONMENT, None);
ctx.connect("environment", "", ".root", "objects", None);

ctx.create("env_attrib", ATTRIBUTES, None);
ctx.connect("env_attrib", "", "environment", "geometryattributes", None);

ctx.set_attribute("env_attrib", &[integer!("visibility.camera", 0)]);

ctx.create("env_shader", SHADER, None);
ctx.connect("env_shader", "", "env_attrib", "surfaceshader", None);

ctx.set_attribute(
    "env_shader",
    &[
        string!(
            "shaderfilename",
            "${DELIGHT}/osl/environmentLight"
        ),
        float!("intensity", 1.5),
    ],
);

ctx.set_attribute(
    "env_shader",
    &[string!("image", "/Users/moritz/code/crates/nsi/assets/wooden_lounge_1k.tdl")],
);

Error: cannot find macro `integer` in this scope

Error: cannot find macro `string` in this scope

Error: cannot find macro `float` in this scope

Error: cannot find macro `string` in this scope

Error: cannot find value `ctx` in this scope

...

Error: cannot find value `ctx` in this scope

Error: cannot find value `SHADER` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `ctx` in this scope

And finally, a camera transform & a [camera](https://nsi.readthedocs.io/en/latest/nodes.html#camera-nodes).

In [15]:
ctx.create("camera_xform", TRANSFORM, None);
ctx.connect("camera_xform", None, ".root", "objects", None);
ctx.set_attribute(
    "camera_xform",
    &[double_matrix!(
        "transformationmatrix",
        &[1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 5., 1.,]
    )],
);

ctx.create("camera", PERSPECTIVE_CAMERA, None);
ctx.connect("camera", None, "camera_xform", "objects", None);
ctx.set_attribute("camera", &[float!("fov", 40.)]);

Error: cannot find macro `double_matrix` in this scope

Error: cannot find macro `float` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `TRANSFORM` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `PERSPECTIVE_CAMERA` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `ctx` in this scope

We hook up a [screen](https://nsi.readthedocs.io/en/latest/nodes.html#the-screen-node) to the camera set the resolution and the antialiasing samples.

We also adjust the shading quality for the whole scene on the [global](https://nsi.readthedocs.io/en/latest/nodes.html#the-global-node) node.

In [16]:
ctx.create("screen", SCREEN, None);
ctx.connect("screen", None, "camera", "screens", None);
ctx.set_attribute(
    "screen",
    &[
        integers!("resolution", &[400, 400]).array_len(2),
        integer!("oversampling", 16),
    ],
);

ctx.set_attribute(
    ".global",
    &[
        integer!("quality.shadingsamples", 64),
    ],
);

Error: cannot find macro `integers` in this scope

Error: cannot find macro `integer` in this scope

Error: cannot find macro `integer` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `SCREEN` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `ctx` in this scope

Error: cannot find value `ctx` in this scope

And we’re ready to render the screen to our notebook.

In [17]:
ctx.as_jupyter("screen");

Error: cannot find value `ctx` in this scope